In [5]:
import boto3
s3_session = boto3.Session()
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')
bucket_name = 'npa02012-main'
s3_bucket = s3_resource.Bucket(bucket_name)

# Env setup (shouldn't be done here)
import os
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3"

In [23]:
for o in s3_bucket.objects.all():
    print(o)

s3.ObjectSummary(bucket_name='npa02012-main', key='kaggle_data/')
s3.ObjectSummary(bucket_name='npa02012-main', key='kaggle_data/titanic.zip')
s3.ObjectSummary(bucket_name='npa02012-main', key='kaggle_data/titanic/gender_submission.csv')
s3.ObjectSummary(bucket_name='npa02012-main', key='kaggle_data/titanic/test.csv')
s3.ObjectSummary(bucket_name='npa02012-main', key='kaggle_data/titanic/train.csv')
s3.ObjectSummary(bucket_name='npa02012-main', key='test/test.txt')


In [3]:
import pyspark
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.util import MLUtils

spark_context = pyspark.SparkContext()
spark_session = pyspark.sql.SparkSession \
    .builder \
    .appName("Spark ML example on titanic data ") \
    .getOrCreate()

In [18]:
aws_creds = s3_session.get_credentials().get_frozen_credentials()
hadoop_conf = spark_context._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3n.impl", "org.apache.hadoop.fs.s3native.NativeS3FileSystem")
hadoop_conf.set("fs.s3n.awsAccessKeyId", aws_creds.access_key)
hadoop_conf.set("fs.s3n.awsSecretAccessKey", aws_creds.access_key)

In [21]:
aws_creds = s3_session.get_credentials().get_frozen_credentials()
hadoop_conf = spark_context._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("fs.s3a.awsAccessKeyId", aws_creds.access_key)
hadoop_conf.set("fs.s3a.awsSecretAccessKey", aws_creds.access_key)

In [22]:
train_path = "s3a://npa02012-main/kaggle_data/titanic/train.csv"

titanic_df = spark_session.read.option('header', 'true')\
                .csv(train_path)
print(titanic_df.count())

Py4JJavaError: An error occurred while calling o56.csv.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2197)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:2654)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2667)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:94)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:2703)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:2685)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:373)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:295)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:46)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:366)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.sca